In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time
import random 

from requests.auth import HTTPBasicAuth

In [2]:
epo_countries = [
    "AL",  # Albania
    "AT",  # Austria
    "BE",  # Belgium
    "BG",  # Bulgaria
    "CH",  # Switzerland
    "CY",  # Cyprus
    "CZ",  # Czech Republic
    "DE",  # Germany
    "DK",  # Denmark
    "EE",  # Estonia
    "ES",  # Spain
    "FI",  # Finland
    "FR",  # France
    "GB",  # United Kingdom
    "GR",  # Greece
    "HR",  # Croatia
    "HU",  # Hungary
    "IE",  # Ireland
    "IS",  # Iceland
    "IT",  # Italy
    "LI",  # Liechtenstein
    "LT",  # Lithuania
    "LU",  # Luxembourg
    "LV",  # Latvia
    "MC",  # Monaco
    "MK",  # North Macedonia
    "MT",  # Malta
    "NL",  # Netherlands
    "NO",  # Norway
    "PL",  # Poland
    "PT",  # Portugal
    "RO",  # Romania
    "RS",  # Serbia
    "SE",  # Sweden
    "SI",  # Slovenia
    "SK",  # Slovakia
    "SM",  # San Marino
    "TR",  # Türkiye
    "US",  # United States
    "CN",  # China
    "JP",  # Japan
    "KR",  # South Korea
    "IN",  # India
    "RU",  # Russia
    "AU",  # Australia
    "BR",  # Brazil
    "CA"   # Canada
]


In [3]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [4]:
def patent_search(date_range, token, start, end):

    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }
    
    params = {
    "q": f'pd={date_range}',
    "range": f"{start}-{end}",
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        
        patents_list = []
        
        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}
        
        xml_data = response.text
        root = ET.fromstring(xml_data)
        
        for patent in root.findall(".//epo:document-id", namespaces=ns):
            country = patent.find("epo:country", namespaces=ns)
            doc_number = patent.find("epo:doc-number", namespaces=ns)
        
            if country is not None and doc_number is not None:
                patents_list.append({
                "country": country.text,
                "doc_number": doc_number.text
            })
        
        return patents_list
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

In [5]:
#print(patent_search("1883", get_token(), 1, 100))

In [14]:
quartiles = {
    "Q1": ("0101", "0331"),
    "Q2": ("0401", "0630"),
    "Q3": ("0701", "0930"),
    "Q4": ("1001", "1231"),
}

all_patents = []
range_size = 100
max_total = 2000 #EPO limit per request


for year in range(1780, 1799):
    token = get_token()
    year_results = []
    
    for quarter, (start_suffix, end_suffix) in quartiles.items():
        start = 1
        end = range_size
        date_range = f"{year}{start_suffix}-{year}{end_suffix}"
        
        while start <= max_total:
            #time.sleep(1.5)
            time.sleep(random.uniform(3, 5))
            query_results = patent_search(date_range, token, start, end)
            #print(query_results)
            if query_results is None:
                print("Error is none")
                break
            if len(query_results) < 100:
                year_results.extend(query_results)
                break
            
            year_results.extend(query_results)
            start += range_size
            end += range_size
    print(year, "has", len(year_results))        
    all_patents.append(year_results)

                
                
        


Error: 404, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>SERVER.EntityNotFound</code>
    <message>No results found</message>
</fault>

Error: 404, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>SERVER.EntityNotFound</code>
    <message>No results found</message>
</fault>

Error: 404, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>SERVER.EntityNotFound</code>
    <message>No results found</message>
</fault>

Error: 404, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>SERVER.EntityNotFound</code>
    <message>No results found</message>
</fault>

1780 has 0
Error: 404, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>SERVER.EntityNotFound</code>
    <message>No results found</message>
</fault>

Error: 404, <?xml version="1.

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
csv_filename = "patents_foster_1700-1799.csv"

with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(["year", "country", "doc_number"])
    
    # Flatten the data while writing
    for i in range (len(all_patents)):
        year = 1780 + i
        patent_list = all_patents[i]

        for patent in patent_list:
            writer.writerow([year, patent["country"], patent["doc_number"]])

print(f"CSV file '{csv_filename}' has been created successfully.")

CSV file 'patents_foster_1700-1799.csv' has been created successfully.
